In [ ]:
!pip install -q malariagen_data
!pip install -q scikit-allel
!pip install -q pomegranate

In [ ]:
import allel
import malariagen_data
import numpy as np
import pandas as pd
import dask
import dask.array as da
# silence some dask warnings
dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.diagnostics.progress import ProgressBar

In [ ]:
import random
import itertools
#import plotly
import plotly.express as px
#import plotly.io as pio
#import pomegranate

#import random
from collections import Counter
from tqdm.dask import TqdmCallback
from tqdm.auto import tqdm
import functools

In [ ]:
# plotting setup
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
import matplotlib_venn as venn
import seaborn as sns

In [ ]:
#Mounting Google Drive
import os
from google.colab import drive
drive.mount("drive")

# make dir
results_dir = "drive/MyDrive/data_dsx/"
os.makedirs(results_dir, exist_ok=True)

Mounted at drive


In [ ]:
ag3 = malariagen_data.Ag3("gs://vo_agam_release/", pre=True)
#ag3

#samples sets
sets = ["AG1000G-BF-A", "AG1000G-BF-B", "AG1000G-BF-C","1191-VO-MULTI-OLOUGHLIN-VMF00106",
        "1191-VO-MULTI-OLOUGHLIN-VMF00140", "1314-VO-BF-KIENTEGA-KIMA-BF-2104"]
df_samples = ag3.sample_metadata(sample_sets=sets)#.set_index("sample_id")
bf_samples = df_samples.query('country == "Burkina Faso"')

In [ ]:
#To access to the genotypes within the 2R chromosomes
chrom2R_gt = ag3.snp_calls(region="2R", sample_sets= sets)

## let's select BF samples
BF_samples = df_samples.eval("(country == 'Burkina Faso')").values
#BF_samples

## To have snp position within the target region in dsx gen
snps_pos = allel.SortedIndex(chrom2R_gt['variant_position'].values)
loc_region = snps_pos.locate_range(48703664 , 48792262)

#To the genomics dataset within the dsx amplicom region and BF_samples
dsx_region = chrom2R_gt.sel(variants=loc_region, samples=BF_samples)
#dsx_region

# To filter the SNP dataset and warp the dataset to GT array
filt = 'gamb_colu_arab'
filt_val = dsx_region[f"variant_filter_pass_{filt}"].values
gt_filtered = allel.GenotypeDaskArray(dsx_region["call_genotype"][filt_val].data)

In [ ]:
## To count the number of alleles
with ProgressBar():
  gt_filtered_arr = allel.GenotypeArray(gt_filtered)
  ac = gt_filtered.count_alleles(max_allele=3).compute()

[########################################] | 100% Completed | 10.74 s
[########################################] | 100% Completed | 7.13 s


In [ ]:
# To get the variants positions within the target region in dsx
is_variant = ac.is_variant()
pos_df = dsx_region['variant_position'][filt_val][is_variant].compute()
pos = allel.SortedIndex(pos_df["variant_position"].values)

# How many segregating sites
print(f'The number of segregating sites is {ac.count_segregating()}.')
print(f'Total variants = {ac.count_variant()}.')
print(f'Biallelic sites = {ac.is_biallelic().sum()}.')
print(f'Multiallelic sites = {ac.count_segregating() - ac.is_biallelic().sum()}.')
print(f'density: {ac.count_segregating()/(loc_region.stop-loc_region.start)}')
print('')
print('')

## diversity
pi = allel.sequence_diversity(snps_pos[loc_region], ac )
D = allel.tajima_d(ac, snps_pos[loc_region])
print(f"Nucleotide diversity = {pi}; \nTajima'D = {D}")

### haplotypes
ht_cohort = allel.HaplotypeArray(gt_filtered_arr.to_haplotypes())
print('')
print('...')
print(f'Number of haplotypes = {ht_cohort.n_haplotypes}.')
print(f'Haplotype diversity = {allel.haplotype_diversity(ht_cohort)}.')
print('done...')

The number of segregating sites is 32243.
Total variants = 32248.
Biallelic sites = 24192.
Multiallelic sites = 8051.
density: 0.3639205860111288


Nucleotide diversity = 0.00637687922994652; 
Tajima'D = -2.3908408075228156

...
Number of haplotypes = 3202.
Haplotype diversity = 0.999999804870472.
done...


In [ ]:
#bf_samples.query('year > 2012').groupby(['location', 'aim_species', 'year']).size()

## define cohorts for computing

In [ ]:
query_year = {'Bana_ara_2014': "location == ['Bana Market', 'Bana Village'] and aim_species == 'arabiensis' and year==2014",
              'Bana_ara_2019': "location == ['Bana Market', 'Bana Village'] and aim_species == 'arabiensis' and year==2019",
              'Bana_col_2012': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2012",
              'Bana_col_2014': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2014",
              'Bana_col_2015': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2015",
              'Bana_col_2016': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2016",
              'Bana_col_2017': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2017",
              'Bana_col_2018': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2018",
              'Bana_col_2019': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2019",
              'Bana_gam_2012': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2012",
              'Bana_gam_2014': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2014",
              'Bana_gam_2015': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2015",
              'Bana_gam_2016': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2016",
              'Bana_gam_2018': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2018",
              'Bana_gam_2019': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2019",
              'Bana_int_2012': "location == ['Bana Market', 'Bana Village'] and aim_species == 'intermediate_gambiae_coluzzii' and year==2012",
              'Mono_gam_2004': "location == 'Monomtenga' and aim_species == 'gambiae' and year==2004",
              'Pala_ara_2014': "location == 'Pala' and aim_species == 'arabiensis' and year==2014",
              'Pala_ara_2015': "location == 'Pala' and aim_species == 'arabiensis' and year==2015",
              'Pala_ara_2016': "location == 'Pala' and aim_species == 'arabiensis' and year==2016",
              'Pala_ara_2018': "location == 'Pala' and aim_species == 'arabiensis' and year==2018",
              'Pala_ara_2019': "location == 'Pala' and aim_species == 'arabiensis' and year==2019",
              'Pala_col_2012': "location == 'Pala' and aim_species == 'coluzzii' and year==2012",
              'Pala_col_2018': "location == 'Pala' and aim_species == 'coluzzii' and year==2018",
              'Pala_col_2019': "location == 'Pala' and aim_species == 'coluzzii' and year==2019",
              'Pala_gam_2012': "location == 'Pala' and aim_species == 'gambiae' and year==2012",
              'Pala_gam_2014': "location == 'Pala' and aim_species == 'gambiae' and year==2014",
              'Pala_gam_2015': "location == 'Pala' and aim_species == 'gambiae' and year==2015",
              'Pala_gam_2016': "location == 'Pala' and aim_species == 'gambiae' and year==2016",
              'Pala_gam_2017': "location == 'Pala' and aim_species == 'gambiae' and year==2017",
              'Pala_gam_2018': "location == 'Pala' and aim_species == 'gambiae' and year==2018",
              'Pala_gam_2019': "location == 'Pala' and aim_species == 'gambiae' and year==2019",
              'Pala_int_2017': "location == 'Pala' and aim_species == 'intermediate_gambiae_coluzzii' and year==2017",
              'Sour_ara_2017': "location == ['Souroukoudinga', 'Souroukoudingan'] and aim_species == 'arabiensis' and year==2017",
              'Sour_ara_2018': "location == 'Souroukoudingan' and aim_species == 'arabiensis' and year==2018",
              'Sour_ara_2019': "location == 'Souroukoudingan' and aim_species == 'arabiensis' and year==2019",
              'Sour_col_2012': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2012",
              'Sour_col_2014': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2014",
              'Sour_col_2015': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2015",
              'Sour_col_2016': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2016",
              'Sour_col_2017': "location == ['Souroukoudinga', 'Souroukoudingan'] and aim_species == 'coluzzii' and year==2017",
              'Sour_col_2018': "location == 'Souroukoudingan' and aim_species == 'coluzzii' and year==2018",
              'Sour_col_2019': "location == 'Souroukoudingan' and aim_species == 'coluzzii' and year==2019",
              'Sour_gam_2012': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2012",
              'Sour_gam_2014': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2014",
              'Sour_gam_2015': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2015",
              'Sour_gam_2016': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2016",
              'Sour_gam_2017': "location == 'Souroukoudingan' and aim_species == 'gambiae' and year==2017",
              'Sour_gam_2018': "location == 'Souroukoudingan' and aim_species == 'gambiae' and year==2018",
              'Sour_gam_2019': "location == 'Souroukoudingan' and aim_species == 'gambiae' and year==2019",
              }

query_sp = {'Bana_ara': "location == ['Bana Market', 'Bana Village'] and aim_species == 'arabiensis'",
            'Bana_col': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii'",
            'Bana_gam': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae'",
            'Bana_int': "location == ['Bana Market', 'Bana Village'] and aim_species == 'intermediate_gambiae_coluzzii'",
            'Mono_gam': "location == 'Monomtenga' and aim_species == 'gambiae'",
            'Pala_ara': "location == 'Pala' and aim_species == 'arabiensis'",
            'Pala_col': "location == 'Pala' and aim_species == 'coluzzii'",
            'Pala_gam': "location == 'Pala' and aim_species == 'gambiae'",
            'Pala_int': "location == 'Pala' and aim_species == 'intermediate_gambiae_coluzzii'",
            'Sour_ara': "location == ['Souroukoudinga', 'Souroukoudingan'] and aim_species == 'arabiensis'",
            'Sour_col': "location == ['Souroukoudinga', 'Souroukoudingan'] and aim_species == 'coluzzii'",
            'Sour_gam': "location == ['Souroukoudinga', 'Souroukoudingan'] and aim_species == 'gambiae'"
            }

pops1 = ['Bana_col_2012','Bana_col_2014','Bana_col_2015','Bana_col_2016','Bana_col_2017','Bana_col_2018','Bana_col_2019',
       'Bana_gam_2012','Bana_gam_2014','Bana_gam_2015','Bana_gam_2016','Mono_gam_2004',
       'Pala_ara_2014','Pala_ara_2015','Pala_ara_2016','Pala_ara_2018','Pala_ara_2019','Pala_col_2012','Pala_gam_2012',
       'Pala_gam_2014','Pala_gam_2015','Pala_gam_2016','Pala_gam_2017','Pala_gam_2018','Sour_col_2019','Sour_col_2012',
       'Sour_col_2014','Sour_col_2015','Sour_col_2016','Sour_col_2017','Sour_col_2019','Sour_gam_2012',
       'Sour_gam_2014','Sour_gam_2015','Sour_gam_2016',]

pops = ['Bana_col', 'Bana_gam', 'Mono_gam', 'Pala_ara', 'Pala_col', 'Pala_gam', 'Sour_col', 'Sour_gam']

## div stats

In [ ]:
dsx_stats = ag3.diversity_stats(sample_query="country == 'Burkina Faso'",
                                cohorts="admin2_year",
                                cohort_size=10,
                                region="AGAP004050",
                                site_mask="gamb_colu_arab",
                                site_class="CDS_DEG_4",
                                sample_sets=sets
                                )

## Segragating sites - functions

In [ ]:
#@functools.lru_cache(maxsize=None)
def compute_snp_stats(contig, sample_query, sample_sets=sets):
  # access genotypes
  ds_geno = ag3.snp_calls(region=contig, sample_query=sample_query, sample_sets=sample_sets)
  #count alleles and load into memory
  filt = 'gamb_colu_arab'
  filt_val = ds_geno[f"variant_filter_pass_{filt}"].values
  pos = ds_geno["variant_position"][filt_val].values
  gt_filtered = allel.GenotypeDaskArray(ds_geno["call_genotype"][filt_val].data)

  # count alleles and load into memory
  with TqdmCallback(desc="Count max alleles"):
    ac = gt_filtered.count_alleles(max_allele=3).compute()
  # compute SNPs
  n_samples = len(bf_samples.query(sample_query))
  n_seg = ac.count_segregating()
  n_var = ac.count_variant()
  dens = n_seg/(48792262-48703664)
  n_biallelic = ac.is_biallelic().sum()
  n_multiallelic = ac.count_segregating() - ac.is_biallelic().sum()
  tx_multiallelic = 100*(ac.count_segregating() - ac.is_biallelic().sum())/ac.count_segregating()
  # Haplotype diversity
  ht_cohort = allel.HaplotypeArray(gt_filtered.to_haplotypes())
  n_hap = ht_cohort.n_haplotypes
  hap_div = allel.haplotype_diversity(ht_cohort)

  # list of stats
  SNPs_list = [n_samples, n_seg, n_var, dens, n_biallelic, n_multiallelic, tx_multiallelic, n_hap, hap_div]

  return pos, SNPs_list


In [ ]:
#compute_snp_stats(contig='AGAP004050', sample_query= query_year['Sour_gam_2019'], sample_sets=sets)

In [ ]:
def compute_snp(query, contig, sample_sets):
  ## def variables
  seg_dict, pos_dict = {}, {}

  ## compute segregating sites
  for key in query.keys():
    pos_dict[key], seg_dict[key] = compute_snp_stats(contig=contig, sample_query= query[key], sample_sets=sample_sets)

  ## Warp to dataframe
  df_tab = pd.DataFrame.from_dict(seg_dict, orient='index')
  df_tab.rename(columns={0: 'n_samples',1:'n_sites',2:'n_variants',3:'var_density',4:'bi_all_sites',5:'multi_all_sites',
                         6:'tx_multi_allelic',7:'n_haplotypes',8:'h_diversity'}, inplace=True)

  return df_tab, pos_dict


In [ ]:
# compute SNP per year per pop
seg_tab_y, pos = compute_snp(query=query_year, contig='2R:48703664-48792262', sample_sets=sets)

# compute SNP per year per pop
seg_tab_pop, pos_pop = compute_snp(query=query_sp, contig='2R:48703664-48792262', sample_sets=sets)

## diversity stats - functions

In [ ]:
def compute_stat(query_dict, pops, samples, cohorts="admin2_year", cohort_size=10, region="2R",
                      site_mask="gamb_colu_arab",site_class="CDS_DEG_4", random_seed=42):

  ## create empty df
  df_list = []

  ## compute diversity stats
  for key in pops:
    if len(samples.query(query_dict[key]))>cohort_size:
      div = ag3.diversity_stats(sample_query=f'{query_dict[key]}',cohorts=cohorts,cohort_size=cohort_size,
                                region=region,site_mask=site_mask,site_class=site_class, random_seed=random_seed)
      ## add population
      site = [key for i in range(div.shape[0])]
      chrom = [region for i in range(div.shape[0])]

      ## insert site and chrom in the df
      div_stat = div.copy()
      div_stat.insert(1, 'site', site)
      div_stat.insert(2, 'chrom', chrom)

      ## append into df list
      df_list.append(div_stat)

  ## concat to df
  stats_df = pd.concat(df_list, ignore_index=True).fillna('')

  return df_list, stats_df

In [ ]:
# diversity statistics in  the dsx gene per year per sp
dlist_dsx_y, stats_dsx_y = compute_stat(query_year, samples=bf_samples, pops=pops1, region='AGAP004050')

# diversity statistics in  the dsx gene per year per sp
dlist_dsx_sp, stats_dsx_sp = compute_stat(query_sp, samples=bf_samples, pops=pops, region='AGAP004050')

## SNP frequencies

In [ ]:
## let's choose a transcripts
ra = 'AGAP004050-RA'
rb = 'AGAP004050-RB'


In [ ]:
## Compute SNP allele frequencies by collection site and species and year
#help(ag3.snp_allele_frequencies)
snps_ra_y = ag3.snp_allele_frequencies(transcript=ra, cohorts=query_year, site_mask='gamb_colu_arab',
                                     sample_sets=sets, drop_invariant=True, effects=True)
snps_rb_y = ag3.snp_allele_frequencies(transcript=rb, cohorts=query_year, site_mask='gamb_colu_arab',
                                     sample_sets=sets, drop_invariant=True, effects=True)

In [ ]:
## Compute SNP allele frequencies by collection site and species
#help(ag3.snp_allele_frequencies)
snps_ra_pop = ag3.snp_allele_frequencies(transcript=ra, cohorts=query_sp, site_mask='gamb_colu_arab',
                                     sample_sets=sets, drop_invariant=True, effects=True)
snps_rb_pop = ag3.snp_allele_frequencies(transcript=rb, cohorts=query_sp, site_mask='gamb_colu_arab',
                                     sample_sets=sets, drop_invariant=True, effects=True)

In [ ]:
## Creat SNP list
snps_list1 = [snps_ra_y, snps_rb_y]
snps_list2 = [snps_ra_pop, snps_rb_pop]

# SNP freq tables
freq_y = pd.concat(snps_list1)
freq_y = freq_y.reset_index().drop_duplicates(subset=['position', 'alt_allele'])
freq_pop = pd.concat(snps_list2)
freq_pop = freq_pop.reset_index().drop_duplicates(subset=['position', 'alt_allele'])

## Times series frequencies

In [ ]:
def ds_freq_tab(ds):
  #extract cohorts into a dataframe
  cohort_vars = [v for v in ds if v.startswith("cohort_")]
  df_cohorts = ds[cohort_vars].to_dataframe()
  df_cohorts.columns = [c.split("cohort_")[1] for c in df_cohorts.columns]

  variant_labels = ds["variant_label"].values
  dfs = []
  for cohort_index, cohort in enumerate(df_cohorts.itertuples()):
    ds_cohort = ds.isel(cohorts=cohort_index)
    dict_df =  {"taxon": cohort.taxon, "area": cohort.area, "date": cohort.period_start,
                "period": str(cohort.period),"sample_size": cohort.size,"position": ds_cohort['variant_position'],
                "ref": ds_cohort['variant_ref_allele'],"alt": ds_cohort['variant_ref_allele'],"aa_change": ds_cohort['variant_aa_change'],
                "variant": variant_labels, "count": ds_cohort["event_count"].values,"nobs": ds_cohort["event_nobs"].values,
                "frequency": ds_cohort["event_frequency"].values, "frequency_ci_low": ds_cohort["event_frequency_ci_low"].values,
                "frequency_ci_upp": ds_cohort["event_frequency_ci_upp"].values
                }
    df = pd.DataFrame(dict_df)
    dfs.append(df)

  df_events = pd.concat(dfs, axis=0).reset_index(drop=True)
  df_events = df_events.query("nobs > 0")

  # Frequencies stats
  frq = df_events["frequency"]
  frq_ci_low = df_events["frequency_ci_low"]
  frq_ci_upp = df_events["frequency_ci_upp"]
  df_events["frequency_error"] = frq_ci_upp - frq
  df_events["frequency_error_minus"] = frq - frq_ci_low

  return df_events

In [ ]:
def plot_freq_time_series(data, height=450, width=900, template='plotly_white', title='{gene} SNP frequencies'):

  #plot time series frequencies
  fig = px.line(data,x="date", y="frequency", error_y="frequency_error",
                error_y_minus="frequency_error_minus", color="variant", markers=True,
                height=height,width=width,template=template, title=title,
                labels={"date": "Years", "frequency": "Allelic frequencies", "variant":'Genetic variants'})

  # figure layout
  fig.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
  fig.update_yaxes(range=[0, 1.0], ticks="outside", col=1)
  fig.update_xaxes(ticks="outside", col=1)

  return fig

In [ ]:
## All populations
ds_aafreq_all = ag3.aa_allele_frequencies_advanced(transcript=ra, area_by="admin1_iso", period_by="year",
                                                   sample_sets=sets, sample_query="country == 'Burkina Faso' and location ==['Bana Market', 'Bana Village', 'Souroukoudinga', 'Souroukoudingan', 'Pala']",
                                                   variant_query="max_af > 0.0001")
freq_events_all = ds_freq_tab(ds_aafreq_all)
#freq_events_all

## Populations from Bana
ds_aafreq_bana = ag3.aa_allele_frequencies_advanced(transcript=ra, area_by="admin1_iso", period_by="year",
                                                    sample_sets=sets, sample_query="country == 'Burkina Faso' and location ==['Bana Market', 'Bana Village']",
                                                    variant_query="max_af > 0.0001")
freq_events_bana = ds_freq_tab(ds_aafreq_bana)
#freq_events_bana

## Populations from Pala
ds_aafreq_pala = ag3.aa_allele_frequencies_advanced(transcript=ra, area_by="admin1_iso", period_by="year",
                                                    sample_sets=sets, sample_query="country == 'Burkina Faso' and location =='Pala'",
                                                    variant_query="max_af > 0.0001")
freq_events_pala = ds_freq_tab(ds_aafreq_pala)

## Populations from Souroukoudinga
ds_aafreq_sr = ag3.aa_allele_frequencies_advanced(transcript=ra, area_by="admin1_iso", period_by="year",
                                                  sample_sets=sets, sample_query="country == 'Burkina Faso' and location ==['Souroukoudinga', 'Souroukoudingan']",
                                                  variant_query="max_af > 0.0001")
freq_events_sr = ds_freq_tab(ds_aafreq_sr)

In [ ]:
#freq_events_all.query('frequency >0.05')

In [ ]:
#freq_events_bana.query('frequency >0.05')

In [ ]:
#freq_events_pala.query('frequency >0.05')

In [ ]:
#freq_events_sr.query('frequency >0.05')

In [ ]:
#ds_aafreq_bana['variant_aa_change']

In [ ]:
#bf_samples.query("country == 'Burkina Faso' and location ==['Bana Market', 'Bana Village', 'Souroukoudinga', 'Souroukoudingan', 'Pala']").groupby(['aim_species']).size()#.sum()

## Save data to csv

In [ ]:
# Save data to csv
## SNP data
seg_tab_y.to_csv('drive/MyDrive/data_dsx/data_saved/seg_tab_y.csv')
seg_tab_pop.to_csv('drive/MyDrive/data_dsx/data_saved/seg_tab_pop.csv')

## diversity data
stats_dsx_y.to_csv('drive/MyDrive/data_dsx/data_saved/stats_dsx_y.csv')
stats_dsx_sp.to_csv('drive/MyDrive/data_dsx/data_saved/stats_dsx_sp.csv')

## SNP freq
freq_y.to_csv('drive/MyDrive/data_dsx/data_saved/freq_y.csv')
freq_pop.to_csv('drive/MyDrive/data_dsx/data_saved/freq_pop.csv')

In [ ]:
## Save time series data
freq_events_all.to_csv('drive/MyDrive/data_dsx/data_saved/freq_events_all.csv')
freq_events_bana.to_csv('drive/MyDrive/data_dsx/data_saved/freq_events_bana.csv')
freq_events_pala.to_csv('drive/MyDrive/data_dsx/data_saved/freq_events_pala.csv')
freq_events_sr.to_csv('drive/MyDrive/data_dsx/data_saved/freq_events_sr.csv')